## World Bank Attribution

In [ ]:
import wbgapi as wb
from scipy.stats import linregress

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import numpy as np

In [ ]:
arcpy.env.workspace = r"C:\Users\olivoshj\Desktop\ArcGIS_Projects\Dam_Project_Aux\Dam_Project_Aux.gdb"

In [ ]:
# World countries' polygons dowloaded from: 
countries_fc = "countries_polygons"
input_ranges = "Extant_Native_Ranges_EE"

In [ ]:
year_columns = ["GDP_" + str(y) for y in range(1960,2024)] + ["PC_" + str(y) for y in range(1960,2024)] + ["GDPG_" + str(y) for y in range(1960,2024)] + ["PCG_" + str(y) for y in range(1960,2024)]
country_fields = " FLOAT; ".join(year_columns) + " FLOAT;" + "EarlyGDP FLOAT;LatestGDP FLOAT;EarlyPC FLOAT;LatestPC FLOAT;GDPslope FLOAT;GDPpercent FLOAT;PCslope FLOAT;PCpercent FLOAT;GDP_yearCo SHORT"
range_fields = "EarlyGDP FLOAT;LatestGDP FLOAT;EarlyPC FLOAT;LatestPC FLOAT;GDPslope FLOAT;GDPpercent FLOAT;PCslope FLOAT;PCpercent FLOAT;GDP_yearCo SHORT"

In [ ]:
arcpy.management.AddFields(countries_fc, country_fields)

In [ ]:
arcpy.management.AddFields(input_ranges, range_fields)

In [ ]:
# search codes for indicators of interest
#wb.series.info()

In [ ]:
# populate country polygons with World Bank indicators for all years available

fields = ['ISO_CC'] + year_columns

with arcpy.da.UpdateCursor(countries_fc, fields) as cursor:
    for country in tqdm(cursor,total=int(arcpy.management.GetCount(countries_fc)[0])):
        try:
            # get yearly GDP
            for yr_gdp in wb.data.fetch('NY.GDP.MKTP.CD', country[0]): # NY.GDP.MKTP.CD	GDP (current US$)
                
                field_name = yr_gdp['time'].replace("YR","GDP_")
        
                country[fields.index(field_name)] = yr_gdp['value']
        
                cursor.updateRow(country)
                
            # get yearly GDP per capita
            for yr_pc in wb.data.fetch('NY.GDP.PCAP.CD', country[0]): # NY.GDP.PCAP.CD = GDP per capita (current US$)
                
                field_name = yr_pc['time'].replace("YR","PC_")
        
                country[fields.index(field_name)] = yr_pc['value']
        
                cursor.updateRow(country)
            
            # get yearly GDP growth
            for yr_gdpg in wb.data.fetch('NY.GDP.MKTP.KD.ZG', country[0]): # NY.GDP.MKTP.KD.ZG = GDP growth (annual %)
                
                field_name = yr_gdpg['time'].replace("YR","GDPG_")
        
                country[fields.index(field_name)] = yr_gdpg['value']
        
                cursor.updateRow(country)
                
            # get GDP per capita growth
            for yr_pcg in wb.data.fetch('NY.GDP.PCAP.KD.ZG', country[0]): #NY.GDP.PCAP.KD.ZG	GDP per capita growth (annual %)
                
                field_name = yr_pcg['time'].replace("YR","PCG_")
        
                country[fields.index(field_name)] = yr_pcg['value']
        
                cursor.updateRow(country)
                
        except:
            continue

In [ ]:
# calculate period statistics for each country
fields = year_columns + ['EarlyGDP','LatestGDP','EarlyPC','LatestPC','GDPslope','GDPpercent','PCslope','PCpercent','GDP_yearCo']
# row ids   [0:-9]          -9          -8         -7         -6        -5            -4         -3        -2            -1

# manually verify column indexes for each time series (gdp, pc, gdpg, pcg)
print(fields[0:64])
print(fields[64:128])
print(fields[128:192])
print(fields[192:-9])

In [ ]:
with arcpy.da.UpdateCursor(countries_fc, fields) as cursor:
    for row in cursor:
        
        gdp_all = [y for y in row[0:64] if y is not None] # collect gdp values for current country
        pc_all = [y for y in row[64:128] if y is not None] # collect gdp per capita
        gdpg_all = [y for y in row[128:192] if y is not None] # collect yearly growth
        pcg_all = [y for y in row[192:-9] if y is not None] # collect yearly growth per capita
        
        if len(gdp_all) > 0:
            row[-9] = gdp_all[0]/1000000 # first gdp value, converted to million USD
            row[-8] = gdp_all[-1]/1000000 # last gdp value, converted to million USD
            row[-7] = pc_all[0] # first gdp pc value (not converted)
            row[-6] = pc_all[-1] # last gdp pc value (not converted)
            row[-5] = linregress(range(0,len(gdp_all)),gdp_all)[0]/1000000 # slope for all gdp years, converted to million USD
            row[-4] = np.mean(gdpg_all) # average per year growth (%)
            row[-3] = linregress(range(0,len(pc_all)),pc_all)[0] # slope for all gdp per capita years (in USD - not converted)
            row[-2] = np.mean(pcg_all) # average percent growth per year in per capita gdp
            row[-1] = len(gdp_all)

            cursor.updateRow(row)

In [ ]:
# select countries with data
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=countries_fc,
    where_clause="GDP_yearCo IS NOT NULL"
)

# project countries to Equal Area reference
arcpy.management.Project(
    in_dataset=countries_fc,
    out_dataset="countries_polygons_prj",
    out_coor_system='PROJCS["WGS_1984_Equal_Earth_Greenwich",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equal_Earth"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]')

In [ ]:
# attribute IUCN ranges with area-weighted means based on proportional overlaps with non-null country values

fields = ['SHAPE@','EarlyGDP','LatestGDP','EarlyPC','LatestPC','GDPslope','GDPpercent','PCslope','PCpercent','GDP_yearCo']

with arcpy.da.UpdateCursor(input_ranges,fields) as cursor:
    
    for row in tqdm(cursor,total=int(arcpy.management.GetCount(input_ranges)[0])):
        
        arcpy.analysis.PairwiseClip(
            in_features="countries_polygons_prj",
            clip_features=row[0],
            out_feature_class="currCountries"
        )

        total_area = sum([c[0] for c in arcpy.da.SearchCursor("currCountries","Shape_Area")])

        for attr in fields[1:]:
            weighted_mean = np.mean([c[0] * c[1] / total_area for c in arcpy.da.SearchCursor("currCountries",[attr,"Shape_Area"]) if c[0] is not None])
            row[fields.index(attr)] = weighted_mean
            cursor.updateRow(row)

In [ ]:
# export table to csv
arcpy.conversion.ExportTable(
    in_table=input_ranges,
    out_table=r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\WorldBank_Attributes.csv",
    field_mapping=f'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99;EarlyGDP "EarlyGDP" true true false 4 Float 0 0,First,#,{input_ranges},EarlyGDP,-1,-1;LatestGDP "LatestGDP" true true false 4 Float 0 0,First,#,{input_ranges},LatestGDP,-1,-1;EarlyPC "EarlyPC" true true false 4 Float 0 0,First,#,{input_ranges},EarlyPC,-1,-1;LatestPC "LatestPC" true true false 4 Float 0 0,First,#,{input_ranges},LatestPC,-1,-1;GDPslope "GDPslope" true true false 4 Float 0 0,First,#,{input_ranges},GDPslope,-1,-1;GDPpercent "GDPpercent" true true false 4 Float 0 0,First,#,{input_ranges},GDPpercent,-1,-1;PCslope "PCslope" true true false 4 Float 0 0,First,#,{input_ranges},PCslope,-1,-1;PCpercent "PCpercent" true true false 4 Float 0 0,First,#,{input_ranges},PCpercent,-1,-1;GDP_yearCount "GDP_yearCount" true true false 2 Short 0 0,First,#,{input_ranges},GDP_yearCo,-1,-1'
)

In [ ]:
# delete intermediate files
arcpy.management.Delete(fr"currCountries;countries_polygons_prj;{arcpy.env.workspace}\currCountries;{arcpy.env.workspace}\countries_polygons_prj")